# Use SAP HANA Cloud vector engine to find the color code
I stored the vector embeddings of city names in a HANA table with each city haaving a corresponding color. Unfortunately I dropped the actual city names, so you will have to use the vector search engine of HANA to find the correct database entry. Once you have an idea which city we are looking for, fill in the city_name parameter and try it out!

In [ ]:
import os
import json

ROOT_PATH_DIR = os.path.dirname(os.getcwd())
AICORE_CONFIG_FILENAME = 'service-key.json'
with open(os.path.join(ROOT_PATH_DIR, AICORE_CONFIG_FILENAME), 'r') as config_file:
    config_data = json.load(config_file)

os.environ["AICORE_AUTH_URL"]=config_data["url"]+"/oauth/token"
os.environ["AICORE_CLIENT_ID"]=config_data["clientid"]
os.environ["AICORE_CLIENT_SECRET"]=config_data["clientsecret"]
os.environ["AICORE_BASE_URL"]=config_data["serviceurls"]["AI_API_URL"]
os.environ["AICORE_RESOURCE_GROUP"]="default"

In [ ]:
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input_text):
    response = embeddings.create(
        input=input_text,      
        model_name='text-embedding-3-small'
    )
    embedding = response.data[0].embedding
    return embedding

# TODO add the city name that we are looking for to get the vector of the city name 
# which you will then use to do the similarity search in the vector database to find the corresponding color code
city_name = ''
if city_name:
    vector = get_embedding(city_name)
    print(vector[1:10])
else:
    print("Please provide a city name.")

In [ ]:
# Establish a secure connection to an SAP HANA database using hdbcli 
from hdbcli import dbapi

cc = dbapi.connect(
    address='b7c3ff95-9e0c-480d-9022-bfaaa268f780.hna0.prod-us10.hanacloud.ondemand.com',
    port='443',
    user='',
    password='',
    autocommit=True,
    sslValidateCertificate=False
    )
cc.isconnected()

In [ ]:
# Select the most similar entries from the CITY_NAMES table based on vector similarity
# This query performs a vector search using cosine similarity
# It returns the top 3 matches, including their ColorCode, and the calculated cosine similarity
# Results are ordered by similarity in descending order (most similar first)
cursor = cc.cursor()
sql_command = f'''SELECT TOP 3 ColorCode,
COSINE_SIMILARITY(Vector, TO_REAL_VECTOR('{vector}')) AS "COS_SIM"
FROM DEV_CHALLENGE_ADMIN.CITY_NAMES
ORDER BY "COS_SIM" DESC;'''

cursor.execute(sql_command)

# Fetch column names
column_names = [description[0] for description in cursor.description]
# Print column names
print(", ".join(column_names))
for row in cursor:
    print(row)

cursor.close()